# Scheme II

## Variables in Scheme
- Variables in scheme have two basic scopes
    - Local scope set with the __let__ command
    - Global scope set with the __define__ command
    - There are a few other variations of let that have slightly different scope
        - We will talk about these when we talk about recursion

## Let
The general syntax of __let__ is

```scheme
(let
    ( (var1 val1) (var2 val2) ... (var_n val_n) )
    (function1)
    (function2)
    .
    .
    .
    (function_n)
)
```

In [1]:
(let 
  (
   (a 10)
   (b 20)
  )
  (+ a b)
)

30

In [2]:
(let
  ((a 10) (b 20) (+ *))
  (+ a b)
)

200

In [10]:
(let
  ((a 10) (b 20) (+ *))
  (+ a b)
)
(+ a b)

unbound-variable: Unbound variable: b

In [11]:
(let
  ((a 10) (b 20) (+ *))
  (display (+ a b))
 (newline)
)
(display (+ 10 20))

200
30

## Nested Let
- When __let__ is nested inside another __let__ the variable can be redefined

In [12]:
(let
  ((a 10) (b 20))
  (let ((b 30))
    (display (+ a b))
    (display 
    "\n")
  )
  (let ((a 20))
    (display (- a b))
  )
)

40
0

## Let Practice
From "The Scheme Programming Language"

Use __let__ to remove common expressions from:
```scheme
(+ (- (* 3 a) b) (+ (* 3 a) b))
(cons (car (list a b c)) (cdr (list a b c)))
```

In [1]:
(define a 20)
(define b 5)
(+ (- (* 3 a) b) (+ (* 3 a) b))

120

In [10]:
(let
 (
  ;(h (list n b))
  (n (* 3 a))

 )
  `
 ;(display h)
 (+ (- n b) (+ n b))
); end let scope

6

In [5]:
(define a 1)
(define b 2)
(define c 3)
(cons (car (list a b c)) (cdr (list a b c)))

(1 2 3)

In [6]:
(let
  (
   (l (list a b c)) 
  )
  (cons (car l) (cdr l))
)

(1 2 3)

## Define
- Define is used to make a variable have global scope
- While it can be used for objects, it is most often used for functions

In [11]:
(define a 20)
(define b 30)
(+ a b)

50

## Functions
- In Scheme, the act of defining a function and naming a function require two separate expressions
- To define a function, a __lambda__ expression is used
    - The general syntax is
```scheme
(lambda (var1 var2 ... varN) 
         expr1 expr2 ... exprN)
```
    - This will return a function (often called a procedure in scheme)

## Naming Functions
- To name a function, use __let__ or __define__
```scheme
(define cadr
    (lambda (x) 
        (car ( cdr x) ) 
    ) 
)
```

In [5]:
(define mystery
  (lambda(x)
   (display (append `(+) x))
   (/ (eval (append `(+) x) user-initial-environment )
      ;(eval (append `(+) x)) ON GL
      (length x) )
  )
)

mystery

In [7]:
(mystery `(1 2 3 4 5 10.01))

(+ 1 2 3 4 5 10.01)

4.168333333333333

In [1]:
(define compose
  (lambda (a b)
    (lambda (x y) 
      (a (b x y) y)
    )
  )
)

compose

In [9]:
((compose + *) 3 2)

8

In [2]:
(define plusStar (compose + *))
(plusStar 3 2)

8

## Function Practice
Write the following functions
- Returns the surface area of a cube given the length of side x, $6x^2$
- A function that converts a temperature given in farenheight to celcius ($\frac{(F - 32) \times 5}{9}$) 

In [3]:
(define sa
 (lambda (x)
   (* x x 6)
 )
)

sa

In [4]:
(sa 2)

24

In [5]:
(define F->C 
 (lambda (f)
   (/
    (*
        (- f 32)
        5)
     9
    )
   
   (* (- f 32)
      (/ 5 9.0))
   (* (- f 32)
      5/9)
 )
)

f->c

In [6]:
(f->c 32)

0

## Variadic  Functions
- Scheme has two ways of creating a function with a variable number of parameters
- Lambda with one argument instead of a list
```scheme
    (lambda var_r
      expr1 expr2 ... exprN
    )
```
- Lambda with an improper list of arguments
```scheme
    (lambda (var1 var2 . var_r)
      expr1 expr2 ... exprN
    )
```

In [7]:
(define vari
  (lambda nums
     (display  nums)
  )
)

vari

In [10]:
(vari `(1 2 3 4) `(5 6 7 8))

((1 2 3 4) (5 6 7 8))

In [6]:
(define improper
  (lambda (x y . z)
     (display  x)
     (newline)
     (display  y)
     (newline)
     (display  z)
  )
)

improper

In [2]:
(improper 1 2 3 4)

1
2
(3 4)

In [3]:
(improper 1 2)

1
2
()

In [4]:
(improper 1 )

wrong-number-of-arguments: The procedure #[compound-procedure 13 improper] has been called with 1 argument; it requires at least 2 arguments.

## Comparison Operators
- The comparison operators that exist in many other languages also exist in Scheme
    - Unlike many languages, these can take multiple arguments, returning true if the correct order is specified
- Examples
    - `(< )`
    - `(string<? )`
    - `(>= )`

In [7]:
(< 1 2)

#t

In [8]:
(< 2 1)

#f

In [9]:
(< 1 2 3 4)

#t

In [10]:
(< 1 2 3 2)

#f

In [11]:
(string<? "abba" "blondie")

#t

In [13]:
(string<? "abbaa" "abba")

#f

In [ ]:
(string<? "abba" "blondie" "carly rae jepsen")
;;Doesn't work in MIT-Scheme :( 
;;Works fine in MzScheme

## Boolean Operators
- Just like Lua, the boolean operators in Scheme behave like function calls, returning a value
- `(and )` returns the first `#f` it finds, or the last element if no `#f` is found
- `(or )` returns the first non `#f` element, returns #f if all are #f

In [1]:
(and #f 3 1 4)

#f

In [2]:
(and 3 1 4)

4

In [3]:
(or #f 1 2 3)

1

In [4]:
(or #f #f #f)

#f

## Conditionals
Scheme has three main conditional statements
- An __if__ statement
- The __cond__ statement
- The __case__ statement

All of these are functions and return something

## If Statement
The syntax of an __if__ statement is
```scheme
(if
    test
    consequent
    alternative ; Optional
)
```

In [5]:
(define max
  (lambda (x y)
    (if (> x y)
        x
        y
    )
  )
)

max

In [6]:
(max 10 11)

11

In [7]:
(max 11 10)

11

In [8]:
(define ifBigger
  (lambda (x y)
    (if (> x y)
        x
    )
  )
)

ifbigger

In [12]:
(ifBigger 10 111)

In [10]:
(ifBigger 10 1)

10

In [13]:
(define canDivide?
  (lambda (n d)
    (if
      (not (= d 0))
      #t
      #f
    )
  )
)

candivide?

In [14]:
(candivide? 20 0)

#f

In [15]:
(canDivide? 20 1)

#t

## Cond Statement
The syntax of a __cond__ statement is
```scheme
(cond
    (test1 expression1 expression2 ...)
    (test2 expressiona expressionb ...)
    .
    .
    .
    (testn expressioni expressionii ...)
    (else expressionI expresionII ....)
)
```

In [9]:
(define grade
  (lambda (score)
    (cond
      ( (>= score 90) "A")
      ( (>= score 80) "B")
      ( (>= score 70) "C")
      ( (>= score 60) "D")
      ( else "F")
    )
  )
)


grade

In [10]:
(grade 20)

"F"

In [7]:
(define whatToWatch
  (lambda (night haveNetflix )
    (cond
      ( (equal? night "Saturday") "Watch SNL")
      ( (equal? night "Tuesday") "Watch ABC")
      ( (equal? night "Thursday") "Watch NBC")
      ( haveNetflix "Watch Master of None")
      ( else "Channel Surf")
    )
  )
)


whattowatch

In [8]:
(whatToWatch "Sunday" #f)

"Channel Surf"

## Case Statements
A case statement is approximently the same as a switch statement in other langauges

The basic snytax is:
```scheme
(case 
    (expression)
    ( (key1 key2 ... keyn) expr1 expr2 .. exprn)
    ( (keyA keyB ... keyN) expra exprb .. exprn)
    ( else expri exprii ... expr_n)
)
```

## Case Statement Example

In [12]:
(define grade2
  (lambda (x)
   (case x
     ( (100 90 91 92 93 94 95 96 97 98 99) "A")
     ( (80 81 82 83 84 85 86 87 88 89) "B")
     ( (70 71 72 73 74 75 76 77 88 89) "C")
     ( (60 61 62 63 64 65 66 67 68 69) "D")
     ( else "F")   
   )
  )
)
(grade2 88)

"B"

## Conditionals Practice
Write a function that does the following:
- Returns if a number is odd or even

In [1]:
(define oddEven?
 (lambda (x)
   (case
        (modulo x 2)
        ((0) "EVEN")
        (else "ODD")
    )
 )
)



oddeven?

In [3]:
(oddEven? 21)

"ODD"

## Loops
- Explicit looping is less commonly used in functional languages, but is still possible
- In Scheme, the looping construct is `do`
```scheme
    (do ( (var1 init1 update1)
          (var2 init2 update2))
        ( (stop-predicate) final-value)
        body1
        body2
    )
```

In [4]:
(do
     (;initialization
      (i 0 ;var and initial value
         (+ 1 i)) ;update rule
     )
     (
      (= 5 i) ; stop-condition
      "Hello" ; Return when stopped
     )
     (display i) ;body
     (newline) ; body
 )

0
1
2
3
4


"Hello"

In [5]:
(define loop 
  (lambda x
    (define z x)
    (do
     (;initialization
      (i 0 ;var and initial value
         (+ 1 i)) ;update rule
     )
     (
      (= (length x) i) ; stop-condition
     )
     (let 
          (
          (y (car z)) 
          )
     (display y) ;body
     (newline) ; body
     )
   )
    (define z (cdr z))
   
)
)

simple-error: Premature reference to reserved name: z

In [ ]:
(loop 6 7 8 9 0 1 2)